In [5]:
function b_n(n::Int64)
    numerator = -n*(n+1)
    denominator = (n+2)*(n+3)
    return numerator/denominator
end

function M_in(size::Int64)
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1, j+1] = 2/(2*i+1) + (b_n(i)*b_n(i))*(2/(2*i+5))
            elseif i==j+2
                fullMatrix[i+1, j+1] = b_n(j)*(2/(2*j+5))
            elseif i==j-2
                fullMatrix[i+1, j+1] = b_n(i)*(2/(2*j+5))
            end
        end
    end
    return fullMatrix
end

function B_in(size::Int64))
    fullMatrix = zeros(size,size)
    for i=0:size-1
        for j=0:size-1
            if i==j
                fullMatrix[i+1,j+1]=2/(2*i+1)
            elseif i==j+2
                fullMatrix[i+1,j+1]=b_n(j)*2/(2*j+5)
            end
        end
    end
    return fullMatrix
end

M_in (generic function with 1 method)

In [6]:
M_in(5)

5×5 Array{Float64,2}:
 2.0   0.0        0.0         0.0        0.0      
 0.0   0.674603   0.0        -0.030303   0.0      
 0.0   0.0        0.42        0.0       -0.0461538
 0.0  -0.047619   0.0         0.314805   0.0      
 0.0   0.0       -0.0666667   0.0        0.257108 